In [1]:
import os
import sys
import argparse
import itertools
import subprocess
import pandas as pd
import multiprocessing
from itertools import cycle, islice
from libs import functions
from datetime import datetime
from multiprocessing import Pool
from subprocess import run, PIPE
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def valid_file(param):
    base, ext = os.path.splitext(param)
    if ext.lower() not in ('.csv', '.fasta','.fa'):
        raise argparse.ArgumentTypeError('File must have a csv or fasta extension')
    return param


def check_arg(args=None):
    parser = argparse.ArgumentParser(description='Avoidance calculator script')
    parser.add_argument('-m', '--mrna',
                        type=valid_file,
                        help='mrna in csv or fasta format',
                        required='True')
    parser.add_argument('-n', '--ncrna',
                        type=valid_file,
                        help='ncrna in csv or fasta format',
                        required='True')
    parser.add_argument('-l','--length',
                        help='length to calculate interactions for. default = 30 nt')
    parser.add_argument('-o', '--output',
                        help='Output file name.',
                        default = 'avoidance')
    parser.add_argument('-p','--processes',
                        type=int,
                        help='number of process to spawn. Default = 16')

    results = parser.parse_args(args)
    return (results.mrna,
            results.ncrna,
            results.length,
            results.output,
            results.processes)


def progress(iteration, total):   
    bars_string = int(float(iteration) / float(total) * 50.)
    sys.stdout.write(
        "\r|%-50s| %d%% (%s/%s)" % (
            '█'*bars_string+ "░" * (50 - bars_string), float(iteration) / float(total) * 100,
            iteration,
            total
        ) 
    )
    sys.stdout.flush()
    if iteration == total:
        print(' Completed!') 
        
        
def fasta_to_dataframe(f):
    fasta_df = pd.read_csv(f,sep='>', lineterminator='>', header=None)
    fasta_df[['accession','sequence']] = fasta_df[0].str.split('\n', 1, expand=True)
    fasta_df['accession'] = '>' + fasta_df['accession']
    fasta_df['sequence'] = fasta_df['sequence'].replace('\n', '', regex=True)
    fasta_df.drop(0, axis=1, inplace=True)
    fasta_df.set_index('accession', inplace=True)
    fasta_df = fasta_df[fasta_df.sequence != '']
    final_df = fasta_df.dropna()
    return final_df


def interaction_calc(seq):
    proc = run(['RNAup', '-b','-o','--interaction_first'], stdout=PIPE,stderr=subprocess.DEVNULL,
               input=str.encode(seq)) #input is a stdin object so encode input str
    return str(proc.stdout).replace("\\n"," ").replace("b'","")

In [ ]:
# %%timeit
mrna = fasta_to_dataframe("test.fa")
ncrna = fasta_to_dataframe("test_ncrna.fa")
mrna['mrna_seq'] = '>' + mrna[1].map(str) + '\n' + mrna[0].map(str).str[:30] + '\n'
ncrna['ncrna_seq'] = '>' + ncrna[1].map(str) + '\n' + ncrna[0].map(str) + '\n'
mrna_seq = [rows['mrna_seq'] for index,rows in mrna.iterrows()]   
ncrna_seq = [rows['ncrna_seq'] for index,rows in ncrna.iterrows()]
index = pd.MultiIndex.from_product([mrna_seq , ncrna_seq], names = ['mrna', 'ncrna'])
sequence_df = pd.DataFrame(index = index).reset_index()
df = sequence_df.pivot(index='mrna',columns='ncrna',values='ncrna')
df['interaction_first'] = df.reset_index().values.sum(axis=1)

In [ ]:
total_pairs = df.shape[0]
my_pool = Pool(4)
interactions = []
functions.progress(0,total_pairs)
for i in my_pool.imap_unordered(interaction_calc, df['interaction_first'], chunksize = int(total_pairs/4)):
    interactions.append(i)
    functions.progress(len(interactions),total_pairs)

my_pool.close()
my_pool.join()

In [ ]:
seq_id = pd.Series(interactions).str.extractall(r'(>[\S]+)')[0].str.replace('>', '', regex=True).to_frame()
ncrna_id = (seq_id.loc[pd.IndexSlice[:, 1:], :]).reset_index().set_index('level_0')
mrna_id = (seq_id.loc[pd.IndexSlice[:, 0], :]).reset_index().set_index('level_0')
binding_energy = pd.Series(interactions).str.extractall(r'(\(-[0-9]+\.[0-9]+)')[0].str.replace('(', '', regex=True).to_frame().reset_index().set_index('level_0')
d = pd.concat([mrna_id.iloc[:,[1]], ncrna_id.iloc[:,[1]]], axis=1)
match = cycle(list(range(len(ncrna))))
d['match'] = [next(match) for i in range(len(d))]

In [ ]:
d = d.reset_index()
d['level_0']=d['level_0'].astype(int)

binding_energy = binding_energy.reset_index()
binding_energy['level_0']=binding_energy['level_0'].astype(int)
binding_energy['match']=binding_energy['match'].astype(int)
d = pd.merge(binding_energy.reset_index(), d.reset_index(), on=['level_0','match'])

In [ ]:
d = d.iloc[:,[5,6,3]]
d.columns = ['Accession', 'ncRNA', 'Binding_energy']

In [3]:
#one ncRNA multi mRNAs
mrna = fasta_to_dataframe("test.fa")
ncrna = fasta_to_dataframe("test_ncrna1.fa")
ncrna = ncrna.reset_index().reset_index().rename(index=str, columns={'index': 'interaction'})

In [4]:
p = 2
label = (x for x in cycle(list(range(0,p))))
label = pd.DataFrame({'label': list(islice(label, len(mrna)))})
df = pd.concat([label, mrna.reset_index()], axis=1)
df['mrna'] = df['accession'] + ':break' + '\n' + df['sequence'] + '\n'
df = df.groupby('label')['mrna'].apply(''.join).to_frame().reset_index()
df.insert(2, 'interaction', 0)
df = pd.merge(df, ncrna, on='interaction')

In [5]:
df['fasta'] = df['accession'] + '\n' + df['sequence'] + '\n' + df['mrna']
fasta = df['fasta'].tolist()

In [6]:
fasta

['>ECD_RS01040_tRNA-Ile_GAT\nAGGCTTGTAGCTCAGGTGGTTAGAGCGCACCCCTGATAAGGGTGAGGTCGGTGGTTCAAGTCCACTCAGGCCTACCA\n>BbCD00385183:break\nATGCTCACTATCGATACGGCGTCCCGCACTTTTCACTGCAAGACCTGGGTGACGGGCCAGACACAGGGCAAGGCGGTCATCGCTCGCGAACGGATCTCGTTCTGGGGCGGCTTCGACCCGCAGACCGGCAGGATCGTGGACCCCTATTCCTCCCTGCGCGGACGCGACATCTCCGATTGCGTCCTGATCCTGCTGTCCTCGAAAGGCTCGTCCGGAACATCGGGCATGCTCTCGCTGGCGACGCGCGCCGGCCACGCGCCCGCGGCGATGATCCAGGTAGAGATGGACCCCGTGACCGTCATGGGCTGTCTCGTCAACAACATTCCCTTGCTGCAGGCGACCGGATTCGATCCGTTCGAGCAGATCCAGGACGGAGACCTGGTGCGGATAGACGGCGAGAACGGTACGATCACCCTCGCCCCCGGGCAAGCCCTCGAGCACCACCACCACCACCACTGA\n>BcCD00540749:break\nATGAACGGGAAGCAAGTAAACAATGTGGAAGGCGTTGTCACAGCGCTTGATAAAAACGGCTTTTATATAGAAGATAATCAGCCGGATAATGATCCAGCTACTTCAGAAGGTATGTACGTATATAAAAAAGATGCGAATGTAGCAGTAGGAGATCTTATTCAAGTTGATGGAGTAGTAGAAGAATATGTTGGACCAGGATATGCAGAGCGATTTGAAACAGACTTAACGACGACAGAAATTAAGGCGAGTCGCGTTGTTGTAATCGCAAAAGATCAATCTTTGCCAGCACCGATTGTACTTGGGGAAAACGGTGTGAAAATCCCAGACCAGATTATTGATAATGATGCATTCGGTTTATTTGATCCAAATGAAGATGCAATCGACTTTTA

In [7]:
my_pool = Pool(p)
interactions = []
progress(0,p)
for i in my_pool.imap_unordered(interaction_calc, fasta):
    interactions.append(i)
    progress(len(interactions), p)

my_pool.close()
my_pool.join()

|██████████████████████████████████████████████████| 100% (2/2) Completed!


In [8]:
print(interactions)

[">ECD_RS01040_tRNA-Ile_GAT >BbCD00385184:break ((((....((((((&))))))..)))) 219,232 :  15,26  (-7.90 = -17.01 + 2.02 + 7.09) GCUCGAGCACCACC&GGUGGUUAGAGC >BcCD00583995:break ((((((&)))))) 320,325 :  15,20  (-7.42 = -10.60 + 0.52 + 2.66) ACCACC&GGUGGU '", ">ECD_RS01040_tRNA-Ile_GAT >BbCD00385183:break ((((((&)))))) 449,454 :  15,20  (-7.54 = -10.60 + 0.40 + 2.66) ACCACC&GGUGGU >BcCD00540749:break ((((((&)))))) 731,736 :  15,20  (-7.33 = -10.60 + 0.61 + 2.66) ACCACC&GGUGGU >BsCD00423013:break ((((((&)))))) 818,823 :  15,20  (-7.44 = -10.60 + 0.51 + 2.66) ACCACC&GGUGGU '"]


In [15]:
#parsing RNAup output
mrna_id = pd.Series(interactions).str.extractall(r'(>[\S]+:break)')[0].str.replace('>', '').str.replace(':break', '').to_frame().reset_index()
ncrna_id = pd.Series(interactions).str.extractall(r'(>[\S]+)')[0].str.replace('>', '').to_frame().loc[pd.IndexSlice[:, 0], :].reset_index()
binding_energy = pd.Series(interactions).str.extractall(r'(\([\S]+\.[0-9]+)')[0].str.replace('(', '').to_frame().reset_index()
d = pd.merge(mrna_id, binding_energy, on=['level_0','match'])

In [20]:
d = pd.merge(ncrna_id, d, on='level_0').iloc[:, [2,4,5]]

In [22]:
d.columns = ['ncRNA', 'Accession', 'binding_energy']

In [23]:
d = d.pivot(index='Accession',columns='ncRNA',values='binding_energy')

In [24]:
d

ncRNA,ECD_RS01040_tRNA-Ile_GAT
Accession,
BbCD00385183,-7.54
BbCD00385184,-7.90
BcCD00540749,-7.33
BcCD00583995,-7.42
BsCD00423013,-7.44
